In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
mcp_server_path = "./match_server.py"
mcp_server_path = os.path.abspath(mcp_server_path)
mcp_server_path

'/home/taohu/workspace/projects/iagent/scratch/match_server.py'

In [4]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(
    model="gemini-2.5-flash-preview-04-17",
    temperature=0.7,
    max_tokens=None,
    max_retries=6
)

In [5]:
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools


server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your math_server.py file
    args=[mcp_server_path],
)



In [ ]:
import asyncio

from iagent.core.messages import HumanMessage, ToolMessage
from langchain.globals import set_debug

set_debug(True)

from IPython.display import display

async def test_mcp():
    async with stdio_client(server_params) as (read, write):
        display(f"Enter stdio_client context")
        async with ClientSession(read, write) as session:
            display(f"Enter ClientSession context")
            await session.initialize()
            tools = await load_mcp_tools(session)
            display(f"Tools:\n{tools}")

            llm_with_tools = llm.bind_tools(tools)
            messages = [HumanMessage(content="what's (3 + 5) x 12?")]
            while True:
                display(f"Messages:{messages}")
                ai_msg = await llm_with_tools.ainvoke(messages)
                messages.append(ai_msg)
                display(f"Response:{ai_msg}")
                if ai_msg.tool_calls:
                    for tool_call in ai_msg.tool_calls:
                        display(f"Tool call: {tool_call}")
                        tool_output = await session.call_tool(tool_call['name'], tool_call['args'])
                        display(f"Tool output: {tool_output}")
                        tool_message = ToolMessage(
                            content=tool_output.content,
                            tool_call_id=tool_call["id"],
                            name=tool_call["name"]
                        )
                        display(f"Tool message: {tool_message}")
                        messages.append(tool_message)
                else:
                    return ai_msg


response = await test_mcp()


'Enter stdio_client context'

'Enter ClientSession context'

In [ ]:

response
